In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from datetime import datetime
from sklearn.model_selection import train_test_split
from tensorflow.keras import Input, Model
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.applications import MobileNetV3Small
#from tensorflow.keras.applications.convnext import ConvNeXtTiny
#from tensorflow.keras.applications.efficientnet import EfficientNetB2, EfficientNetB3
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet201
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
#from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.resnet50 import preprocess_input
#from tensorflow.keras.applications.resnet_rs import ResNetRS50
#from tensorflow.keras.applications.resnet_v2 import ResNet50V2
#from tensorflow.keras.applications.vgg16 import VGG16
#from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Add, AveragePooling2D, BatchNormalization, Conv2D, Dense, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, Layer, MaxPool2D, ReLU, Resizing
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import L2
from tqdm import tqdm

In [ ]:
# mount your own google drive, after running the code, sign in with your own google account
# To load the project dataset, you should add a shortcut of the project folder in your own google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# paths
proj_path = '/content/drive/MyDrive/DSA5204_FAP_Project'
dataset_path = proj_path + '/Datasets/SCUT-FBP'
image_path = dataset_path + '/Images/'
rate_path = dataset_path + '/Rating_Collection' 

In [ ]:
print(tf.__version__)
print("Num GPUs Available", len(tf.config.experimental.list_physical_devices('GPU')))

2.12.0
Num GPUs Available 1


## 1. Clean data

In [ ]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
os.mkdir('/Cleaned_Images/')

In [ ]:
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_selfie_segmentation = mp.solutions.selfie_segmentation

file_list = glob.glob(image_path + '/*.jpg')

BG_COLOR = (255, 255, 255)
with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
    for i in tqdm(range(len(file_list))):
        file_name = file_list[i].split('/')[-1]
        image = cv2.imread(file_list[i])
        image_height, image_width, _ = image.shape

        results = selfie_segmentation.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask,) * 3, axis=-1) > 0.1
        
        #fg_image = np.zeros(image.shape, dtype=np.uint8)
        #fg_image[:] = MASK_COLOR
        bg_image = np.zeros(image.shape, dtype=np.uint8)
        bg_image[:] = BG_COLOR

        output_image = np.where(condition, image, bg_image)
        
        # cleaned images are under folder /DSA5204_FAP_Project/Datasets/SCUT-FBP/Cleaned_Images
        cv2.imwrite( '/Cleaned_Images/' + file_name, output_image)

100%|██████████| 5510/5510 [02:47<00:00, 32.82it/s]


## 2. Load data

In [ ]:
image_path='/Cleaned_Images/'

In [ ]:
# load data: y_arr = [rating, race, gender]
def load_data(img_dir, label_dir):
  all_ratings = pd.read_csv(label_dir, sep = ' ', header = None)
  all_ratings.columns = ['img_path', 'rating']

  img_arr = np.zeros([len(all_ratings), 112, 112, 3]) # because of RAM limit
  y_arr = np.zeros([len(all_ratings), 3])

  for i in tqdm(range(len(all_ratings))):
  #for i in tqdm(range(500)):
    file_name = all_ratings.iloc[i, 0]
    race = file_name[0]
    gender = file_name[1]
    if race == 'A':
      y_arr[i, 1] = 0
    else:
      y_arr[i, 1] = 1

    if gender == 'M':
      y_arr[i, 2] = 0
    else:
      y_arr[i, 2] = 1 
        
    y_arr[i, 0] = all_ratings.iloc[i, 1]
    
    img = tf.io.read_file(img_dir + file_name)
    img = tf.image.decode_jpeg(img, channels = 3)
    img = tf.keras.layers.Resizing(112, 112)(img)
    img = preprocess_input(img)
    img_arr[i] = img

  return img_arr, y_arr

img_arr, y_arr = load_data(image_path, rate_path + '/All_labels.txt')

100%|██████████| 5500/5500 [00:38<00:00, 144.70it/s]


In [ ]:
#np.save('/content/drive/MyDrive/img_arr.npy', img_arr)
#np.save('/content/drive/MyDrive/y_arr.npy', y_arr)

In [ ]:
#img_arr = np.load('/content/drive/MyDrive/img_arr.npy')
#y_arr = np.load('/content/drive/MyDrive/y_arr.npy')

In [ ]:
img_full_train, img_test, y_full_train, y_test = train_test_split(img_arr, y_arr, stratify = y_arr[:, 2], test_size = 0.2, 
                                                                  random_state = 0)
img_train, img_val, y_train, y_val = train_test_split(img_full_train, y_full_train, stratify = y_full_train[:, 2], 
                                                      test_size = 0.2, random_state = 0)

rating_train = y_train[:, 0]
rating_val = y_val[:, 0]
rating_test = y_test[:, 0]

gender_train = y_train[:, 1]
gender_val = y_val[:, 1]
gender_test = y_test[:, 1]

## 3. Model

In [ ]:
class Custom_Dropout(Layer):
    def __init__(self, rate, **kwargs):
        super(Custom_Dropout, self).__init__(**kwargs)
        self.rate = rate

        def call(self, inputs, training = None):
            if training:
                return tf.nn.dropout(inputs, rate = self.rate)
            return inputs

        
        
class InceptionResNetV2_MT_Extension(Model):
    def __init__(self):
        super(InceptionResNetV2_MT_Extension, self).__init__()
        
        self.inception = InceptionResNetV2(input_shape=(112, 112, 3),include_top=False,weights='imagenet')
        self.inception.trainable = False

        self.fc1_rating = Dense(500, kernel_regularizer = L2(0.0005))
        self.bn_fc1_rating = BatchNormalization()
        self.fc2_rating = Dense(1, kernel_regularizer = L2(0.0005))
        
        self.fc1_gender = Dense(500, kernel_regularizer = L2(0.0005))
        self.bn_fc1_gender = BatchNormalization()
        self.fc2_gender = Dense(1, kernel_regularizer = L2(0.0005), activation = 'sigmoid')
        
        
        self.global_avg_pool = GlobalAveragePooling2D()
        self.flatten = Flatten()
        self.dropout = Custom_Dropout(0.3)
        
        
        
    def call(self, inputs, training = None):
        x = self.global_avg_pool(self.inception(inputs))
        x = self.flatten(x)
        
        x_rating = self.fc2_rating(self.dropout(self.bn_fc1_rating(self.fc1_rating(x))))
        x_gender = self.fc2_gender(self.dropout(self.bn_fc1_gender(self.fc1_gender(x))))
        
        return x_rating, x_gender

In [ ]:
Loss_Functions = {'output_1': 'mse', 'output_2': 'binary_crossentropy'}
Loss_Weights = {'output_1': 5, 'output_2': 1}

In [ ]:
def custom_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tf.math.exp(-0.01) 

    
    
required_callbacks = [
    #tf.keras.callbacks.ModelCheckpoint(filepath = 'checkpoint', monitor = 'val_loss', save_best_only = True),
    #tf.keras.callbacks.TensorBoard(),
    tf.keras.callbacks.EarlyStopping(patience = 30, restore_best_weights = True),
    tf.keras.callbacks.LearningRateScheduler(custom_scheduler),
    tf.keras.callbacks.ReduceLROnPlateau(factor = 0.5, patience = 10, verbose = 1, cooldown = 5, min_lr = 0.0000001),
    tf.keras.callbacks.TerminateOnNaN()
]

In [ ]:
adam_optimizer = Adam(learning_rate = 0.005)
model = InceptionResNetV2_MT_Extension()
model.compile(optimizer = adam_optimizer, loss = Loss_Functions, loss_weights = Loss_Weights)

In [ ]:
model.fit(img_train, [rating_train, gender_train], batch_size = 25, epochs = 1000, verbose = 1, 
          callbacks = required_callbacks, validation_data = (img_val, [rating_val, gender_val]))

Epoch 1/1000
141/141 [==============================] - 35s 98ms/step - loss: 20.0026 - output_1_loss: 3.6983 - output_2_loss: 0.5554 - val_loss: 3.6931 - val_output_1_loss: 0.4670 - val_output_2_loss: 0.5388 - lr: 0.0050
Epoch 2/1000
141/141 [==============================] - 7s 52ms/step - loss: 3.4444 - output_1_loss: 0.4558 - output_2_loss: 0.4449 - val_loss: 4.1768 - val_output_1_loss: 0.4793 - val_output_2_loss: 1.1463 - lr: 0.0050
Epoch 3/1000
141/141 [==============================] - 7s 51ms/step - loss: 3.4776 - output_1_loss: 0.4900 - output_2_loss: 0.4448 - val_loss: 9.5053 - val_output_1_loss: 0.9566 - val_output_2_loss: 4.1675 - lr: 0.0050
Epoch 4/1000
141/141 [==============================] - 7s 52ms/step - loss: 3.2796 - output_1_loss: 0.4616 - output_2_loss: 0.4621 - val_loss: 11.4073 - val_output_1_loss: 2.0238 - val_output_2_loss: 0.7999 - lr: 0.0050
Epoch 5/1000
141/141 [==============================] - 7s 53ms/step - loss: 3.3822 - output_1_loss: 0.5008 - output_

In [ ]:
results = model.evaluate(img_test, [rating_test, gender_test])

35/35 [==============================] - 3s 68ms/step - loss: 2.4002 - output_1_loss: 0.3994 - output_2_loss: 0.3663


In [ ]:
np.sqrt(results[1])

0.6319892143162128

In [ ]:
model_path = '/content/drive/MyDrive/Inception_Resnet_V2'
model.save(model_path)

## 4. Evaluate

In [ ]:
#model = tf.keras.models.load_model(model_path)

In [ ]:
scut_dir = '/content/drive/MyDrive/SCUT-FBP/'
mebeauty_dir = '/content/drive/MyDrive/MEBeauty/'

In [ ]:
import mediapipe as mp

### 4.1 SCUT-FBP Test Dataset Preparation and Evaluation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_excel(scut_dir + 'Rating_Collection/Rating_Collection/Attractiveness label.xlsx', header = 0)
img_arr = np.zeros([len(rating_df), 112, 112, 3])
y_arr = np.zeros([len(rating_df), 2])

for i in tqdm(range(len(rating_df))):
    num = rating_df.iloc[i, 0]
    img = cv2.imread(scut_dir + 'Data_Collection/SCUT-FBP-' + str(num) + '.jpg')
    
    with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
        results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
        bg_image = np.zeros(img.shape, dtype = np.uint8)
        bg_image[:] = (255, 255, 255)
        cleaned_img = np.where(condition, img, bg_image)
    
    cleaned_img = tf.keras.layers.Resizing(112, 112)(cleaned_img)
    cleaned_img /= 255
    img_arr[i] = cleaned_img
    
    y_arr[i, 0] = rating_df.iloc[i, 1]
    y_arr[i, 1] = 1
    
#np.save(scut_dir + 'img_arr.npy', img_arr)
#np.save(scut_dir + 'y_arr.npy', y_arr)

100%|██████████| 500/500 [00:25<00:00, 19.69it/s]


In [ ]:
#img_arr = np.load(scut_dir + 'img_arr.npy')
#y_arr = np.load(scut_dir + 'y_arr.npy')

In [ ]:
results = model.evaluate(img_arr, [y_arr[:, 0], y_arr[:, 1]])

16/16 [==============================] - 1s 53ms/step - loss: 7.9874 - output_1_loss: 0.6555 - output_2_loss: 4.6732


In [ ]:
print('RMSE: '+str(np.sqrt(results[1])))

RMSE: 0.8096110997434255


### 4.2 MEBeauty Test Dataset Preparation

In [ ]:
mp_selfie_segmentation = mp.solutions.selfie_segmentation

rating_df = pd.read_csv(mebeauty_dir + 'landmarks.csv', header = 0)
#img_arr = np.zeros([len(rating_df), 224, 224, 3])
#y_arr = np.zeros([len(rating_df), 2])

img_list = []
y_list = []

for i in tqdm(range(len(rating_df))):
    file_path = rating_df.iloc[i, 1]
    file_name = '/'.join(file_path.split('/')[5:])
    gender = file_path.split('/')[5]   
    try:
        img = cv2.imread(mebeauty_dir + 'cropped_images/images_crop_align_mtcnn/' + file_name)
        '''
        with mp_selfie_segmentation.SelfieSegmentation(model_selection = 0) as selfie_segmentation:
            results = selfie_segmentation.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            condition = np.stack((results.segmentation_mask, ) * 3, axis = -1) > 0.1
            bg_image = np.zeros(img.shape, dtype = np.uint8)
            bg_image[:] = (255, 255, 255)
            cleaned_img = np.where(condition, img, bg_image)
        '''

        img = tf.keras.layers.Resizing(112, 112)(img)
        img /= 255
        #img_arr[i] = img
        img = img[np.newaxis, :, :, :]
        img_list.append(img)
        
        if gender == 'female':
            #y_arr[i, 1] = 1
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 1]).reshape(1, 2))
        else:
            #y_arr[i, 1] = 0
            y_list.append(np.array([rating_df.iloc[i, 2] / 2, 0]).reshape(1, 2))
            
        #y_arr[i, 0] = rating_df.iloc[i, 2]
    except:
        continue
    
img_arr = np.concatenate(img_list, axis = 0)
y_arr = np.concatenate(y_list, axis = 0)
    
#np.save(mebeauty_dir + 'img_arr.npy', img_arr)
#np.save(mebeauty_dir + 'y_arr.npy', y_arr)

100%|██████████| 2459/2459 [23:50<00:00,  1.72it/s]


In [ ]:
#img_arr = np.load(mebeauty_dir + 'img_arr.npy')
#y_arr = np.load(mebeauty_dir + 'y_arr.npy')